# IMPORTS

## Libraries

In [20]:
import pandas as pd
import math

## Helper Functions

## Loading Data

In [4]:
salesRaw = pd.read_csv('../01-Data/train.csv', low_memory=False)
storeRaw = pd.read_csv('../01-Data/store.csv', low_memory=False)

### Merge Datasets

In [5]:
dfRaw = salesRaw.merge(storeRaw, how='left', on='Store')

# DESCRIPTION OF THE DATA

## Columns

In [7]:
dfRaw1 = dfRaw.copy()

In [8]:
dfRaw1.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

## Data Dimensions

In [15]:
print(f'Number of Rows: {dfRaw1.shape[0]}')
print(f'Number of Columns: {dfRaw1.shape[1]}')

Number of Rows: 1017209
Number of Columns: 18


## Data Types

In [16]:
dfRaw1.dtypes

Store                          int64
DayOfWeek                      int64
Date                          object
Sales                          int64
Customers                      int64
Open                           int64
Promo                          int64
StateHoliday                  object
SchoolHoliday                  int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object

In [28]:
dfRaw1['Date'] = pd.to_datetime(dfRaw1['Date'])

## Not a Number

### Sum

In [17]:
dfRaw1.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

### Mean

In [18]:
dfRaw1.isnull().mean()

Store                        0.000000
DayOfWeek                    0.000000
Date                         0.000000
Sales                        0.000000
Customers                    0.000000
Open                         0.000000
Promo                        0.000000
StateHoliday                 0.000000
SchoolHoliday                0.000000
StoreType                    0.000000
Assortment                   0.000000
CompetitionDistance          0.002597
CompetitionOpenSinceMonth    0.317878
CompetitionOpenSinceYear     0.317878
Promo2                       0.000000
Promo2SinceWeek              0.499436
Promo2SinceYear              0.499436
PromoInterval                0.499436
dtype: float64

## Fillout NA

In [19]:
maxValueCompetitionDistance = dfRaw1['CompetitionDistance'].max()

In [31]:
dfRaw1.sample(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,MonthMap
899194,175,2,2013-04-16,4088,608,1,0,0,0,c,a,4130.0,4.0,2013.0,0,16.0,16.0,0,Apr
730724,70,6,2013-09-14,5339,548,1,0,0,0,c,c,4840.0,9.0,2013.0,0,37.0,37.0,0,Sep
304800,207,7,2014-10-19,0,0,0,0,0,0,a,a,6870.0,10.0,2014.0,0,42.0,42.0,0,Oct
39552,528,5,2015-06-26,9445,896,1,0,0,0,a,c,20620.0,6.0,2015.0,1,37.0,2009.0,"Jan,Apr,Jul,Oct",Jun
482040,31,5,2014-04-25,4924,526,1,0,0,1,d,c,9800.0,7.0,2012.0,0,17.0,17.0,0,Apr


In [32]:
# CompetitionDistance
    #distance in meters to the nearest competitor store
maxValueCompetitionDistance = dfRaw1['CompetitionDistance'].max()
dfRaw1['CompetitionDistance'] = dfRaw1['CompetitionDistance'].apply(lambda row: maxValueCompetitionDistance*100 if math.isnan(row) else row)


# CompetitionOpenSinceMonth
    #gives the approximate month of the time the nearest competitor was opened
dfRaw1['CompetitionOpenSinceMonth'] = dfRaw1.apply(lambda row: row['Date'].month if math.isnan(row['CompetitionOpenSinceMonth']) else row['CompetitionOpenSinceMonth'], axis=1)


# CompetitionOpenSinceYear
    # gives the approximate year of the time the nearest competitor was opened
dfRaw1['CompetitionOpenSinceYear'] = dfRaw1.apply(lambda row: row['Date'].year if math.isnan(row['CompetitionOpenSinceYear']) else row['CompetitionOpenSinceYear'], axis=1)


# Promo2SinceWeek
    #describes the calendar week when the store started participating in Promo2
dfRaw1['Promo2SinceWeek'] = dfRaw1.apply(lambda row: row['Date'].week if math.isnan(row['Promo2SinceWeek']) else row['Promo2SinceWeek'], axis=1)


# Promo2SinceYear
    #describes the year when the store started participating in Promo2
dfRaw1['Promo2SinceYear'] = dfRaw1.apply(lambda row: row['Date'].week if math.isnan(row['Promo2SinceYear']) else row['Promo2SinceYear'], axis=1)


# PromoInterval
    #describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew.\
    #E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store
monthMap = {
                1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
            }

dfRaw1['PromoInterval'].fillna(0, inplace=True)
dfRaw1['MonthMap'] = dfRaw1['Date'].dt.month.map(monthMap)

dfRaw1['IsPromo'] = dfRaw1[['PromoInterval', 'MonthMap']].apply(lambda row: 0 if row['PromoInterval'] == 0 else 1 if row['MonthMap'] in row['PromoInterval'].split(',') else 0, axis=1)

## Change Types

In [33]:
dfRaw1.dtypes

Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                  int64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth           float64
CompetitionOpenSinceYear            float64
Promo2                                int64
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
MonthMap                             object
IsPromo                               int64
dtype: object

In [34]:
dfRaw1['CompetitionOpenSinceMonth'] = dfRaw1['CompetitionOpenSinceMonth'].astype(int)
dfRaw1['CompetitionOpenSinceYear'] = dfRaw1['CompetitionOpenSinceYear'].astype(int)
dfRaw1['Promo2SinceWeek'] = dfRaw1['Promo2SinceWeek'].astype(int)
dfRaw1['Promo2SinceYear'] = dfRaw1['Promo2SinceYear'].astype(int)

## Descriptive Statistical